In [1]:
import pandas as pd

In [2]:
drug = pd.read_csv("../data/eICU/infusiondrug.csv")
diagnosis = pd.read_csv("../data/eICU/diagnosis.csv")
patients_weight = pd.read_csv("../data/eICU/patient.csv")[["patientunitstayid", "admissionweight"]]
patient = pd.read_csv("../data/eICU/patient.csv")
drug = drug.merge(patients_weight, on="patientunitstayid")

C:\Users\eythe\AppData\Local\Temp\ipykernel_9120\3265378624.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  drug = pd.read_csv("../data/eICU/infusiondrug.csv")


In [3]:
# add hospitalID and unitType and wardID to the drug table corelated to patientunitstayid
drug["hospitalid"] = drug["patientunitstayid"].map(patient.set_index("patientunitstayid")["hospitalid"])
drug["unittype"] = drug["patientunitstayid"].map(patient.set_index("patientunitstayid")["unittype"])
drug["wardid"] = drug["patientunitstayid"].map(patient.set_index("patientunitstayid")["wardid"])

In [4]:
# filter drug from NA in drugname, patientunitstayid, hospitalid, unittype, wardid
drug = drug[~drug['drugname'].isna()]
drug = drug[~drug['patientunitstayid'].isna()]
drug = drug[~drug['hospitalid'].isna()]
drug = drug[~drug['unittype'].isna()]
drug = drug[~drug['wardid'].isna()]

#filter drug to include only drugnames that start with "Norepinephrine"
drug = drug[drug['drugname'].str.startswith("Norepinephrine")]

In [5]:
# sort drug by patientunitstayid and drugstartoffset
drug = drug.sort_values(by=["patientunitstayid", "infusionoffset"])
drug

,infusiondrugid,patientunitstayid,infusionoffset,drugname,drugrate,infusionrate,drugamount,volumeoffluid,patientweight,admissionweight,hospitalid,unittype,wardid
4,1969910,242082,55,Norepinephrine (mcg/min),2.13,8.0,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
2,1968206,242082,125,Norepinephrine (mcg/min),7,26.3,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
10,1985898,242082,230,Norepinephrine (mcg/min),9.07,34.0,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
12,1972141,242082,275,Norepinephrine (mcg/min),10,37.5,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
1,1998443,242082,425,Norepinephrine (mcg/min),10.93,41.0,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4803535,75122092,3353251,3823,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,15,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104
4803681,75308975,3353251,3968,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,16,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104
4803654,75246300,3353251,4043,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,12,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104
4803592,75148203,3353251,4358,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,10,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104


In [6]:
drug.groupby(["hospitalid", "unittype"])["patientunitstayid"].nunique().sort_values()

hospitalid  unittype    
342         Med-Surg ICU      1
151         Med-Surg ICU      1
265         MICU              1
156         Med-Surg ICU      1
438         Med-Surg ICU      1
                           ... 
157         Med-Surg ICU    375
167         Med-Surg ICU    384
188         MICU            430
300         Med-Surg ICU    452
199         Med-Surg ICU    525
Name: patientunitstayid, Length: 195, dtype: int64

In [8]:
bp = pd.read_csv("../preprocess/filtered_bp_eicu.csv")
bp = bp.sort_values(by=["stay_id", "cur_bp_time"])
bp = bp[bp["stay_id"].isin(drug["patientunitstayid"])]
bp["next_bp_time"] = bp.groupby("stay_id")["cur_bp_time"].shift(-1)
bp["interval"] = bp["next_bp_time"] - bp["cur_bp_time"]
drug["drugrate"] = pd.to_numeric(drug["drugrate"], errors='coerce')
bp["age"] = pd.to_numeric(bp["stay_id"].map(patient.set_index("patientunitstayid")["age"]), errors='coerce')
bp["hospitalid"] = bp["stay_id"].map(patient.set_index("patientunitstayid")["hospitalid"])
bp["unittype"] = bp["stay_id"].map(patient.set_index("patientunitstayid")["unittype"])
bp["wardid"] = bp["stay_id"].map(patient.set_index("patientunitstayid")["wardid"])
# create a new column for the number of patients per hospital
# bp["pat_per_hos"] = bp["hospitalid"].map(bp.groupby(["hospitalid"]).agg({"stay_id": ["nunique"]}).sort_values(by=("stay_id", "nunique")))


In [9]:
# find the number of patients per hospital
bp.groupby(["hospitalid"]).agg({"stay_id": ["nunique"]}).sort_values(by=("stay_id", "nunique"))


,stay_id
,nunique
hospitalid,
279,1
422,1
174,1
404,1
182,1
...,...
157,155
148,156


In [10]:
bp["pat_per_hos"]

KeyError: 'pat_per_hos'

In [11]:
# give me a table of hospitals, wardid and number of patients per hospital
bp_hospital = bp.groupby(['hospitalid']).agg({'stay_id': ['nunique']})

In [12]:
bp["stay_id"].unique()

array([ 243334,  243643,  244256, ..., 3352721, 3353144, 3353251],
      dtype=int64)

In [13]:
print(f'Number of patients in the dataset: {bp_big["stay_id"].nunique()}')
print(f'Number of hospitals in the dataset: {bp_big["hospitalid"].nunique()}')
print(f'Number of patients per hospital in the dataset: {bp_big["pat_per_hos"].unique()}')

NameError: name 'bp_big' is not defined

In [14]:
# bp_big will consist of patients from hospitalid with more then 400 differnt patientsunitstayid
bp_big = bp[bp["hospitalid"].isin(drug.groupby("hospitalid")["patientunitstayid"].nunique().sort_values()[drug.groupby("hospitalid")["patientunitstayid"].nunique().sort_values() > 400].index)]
bp_big

,Unnamed: 0,stay_id,cur_bp_time,cur_bp,next_bp_time,interval,age,hospitalid,unittype,wardid
821904,1679952,426975,1051,63.0,1056.0,5.0,69.0,142,Med-Surg ICU,256
821898,1679936,426975,1056,62.0,1061.0,5.0,69.0,142,Med-Surg ICU,256
821888,1679920,426975,1061,64.0,1066.0,5.0,69.0,142,Med-Surg ICU,256
822021,1680165,426975,1066,71.0,1071.0,5.0,69.0,142,Med-Surg ICU,256
822016,1680149,426975,1071,77.0,1076.0,5.0,69.0,142,Med-Surg ICU,256
...,...,...,...,...,...,...,...,...,...,...
2656663,672834,3130750,4540,199.0,4545.0,5.0,70.0,420,SICU,1027
2656622,672741,3130750,4545,199.0,4550.0,5.0,70.0,420,SICU,1027
2656630,672757,3130750,4550,199.0,4555.0,5.0,70.0,420,SICU,1027
2656041,670562,3130750,4555,199.0,4560.0,5.0,70.0,420,SICU,1027


In [15]:
bp_big["stay_id"].nunique()

1637

In [16]:
# create a plot of bp over time for each patient using plotly subplot and add to each patient plot the norepinephrine infusion from drug table as scatter plot
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np
for i in range(100):
    # assign random patient to pat
    pat = bp_big["stay_id"].value_counts()[bp_big["stay_id"].value_counts() > 0].sample(5).index[0]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=bp_big[bp_big["stay_id"] == pat]["cur_bp_time"], y=bp_big[bp_big["stay_id"] == pat]["cur_bp"], name="MAP"))
    # scatter plot of norepinephrine infusion, not line plot
    # fig.add_trace(go.Scatter(x=drug[drug["patientunitstayid"] == pat]["infusionoffset"], y=drug[drug["patientunitstayid"] == pat]["drugrate"], name="drug", mode="markers", marker=dict(color="red")), row=i+1, col=1)
    x = drug[drug["patientunitstayid"] == pat]["infusionoffset"]
    z = drug[drug["patientunitstayid"] == pat]["drugrate"]
    # make z values between 0 and 10
    z2 = (z - z.min()) / (z.max() - z.min()) * 10
    # set y as a liitle under the min of bp and higher drugrate be higher
    y = bp_big[bp_big["stay_id"] == pat]["cur_bp"].min() - 10 + z2
    fig.add_trace(go.Scatter(x=x, y=y * np.ones(x.size), name="NOR", mode="markers", marker=dict(color=z,colorscale='Viridis',
        colorbar=dict(title='NOR')
    ),
    )) # TODO added drug amount as color
    fig.update_layout(title_text="bp over time for 5 patients")
    fig.write_image(f"bp{i}.png")
    # create a histogram of bp values for the patient
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=bp_big[bp_big["stay_id"] == pat]["cur_bp"], name="bp"))
    fig.update_layout(title_text="bp over time for patient")
    fig.write_image(f"bp_hist{i}.png")



ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [17]:
!pip install -U kaleido

     ---------------------------------------- 0.0/65.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/65.9 MB 1.3 MB/s eta 0:00:52
     --------------------------------------- 0.1/65.9 MB 907.3 kB/s eta 0:01:13
     ---------------------------------------- 0.2/65.9 MB 1.2 MB/s eta 0:00:54
     ---------------------------------------- 0.2/65.9 MB 1.5 MB/s eta 0:00:44
     ---------------------------------------- 0.4/65.9 MB 1.6 MB/s eta 0:00:41
     ---------------------------------------- 0.5/65.9 MB 1.8 MB/s eta 0:00:36
     ---------------------------------------- 0.7/65.9 MB 2.4 MB/s eta 0:00:28
      --------------------------------------- 1.0/65.9 MB 2.9 MB/s eta 0:00:23
     - -------------------------------------- 1.9/65.9 MB 4.6 MB/s eta 0:00:14
     - -------------------------------------- 2.8/65.9 MB 6.0 MB/s eta 0:00:11
     -- ------------------------------------- 3.4/65.9 MB 6.8 MB/s eta 0:00:10
     -- ------------------------------------- 3.9/65.9 MB 